In [1]:
import praw
import csv
import writeToS3 as s3
from os import listdir
from os.path import isfile, join
import pandas as pd

In [2]:
files = [join('Parenting', f) for f in listdir('Parenting') if isfile(join('Parenting', f))]
print(files)

['Parenting\\Parenting_2008-10.csv', 'Parenting\\Parenting_2008-11.csv', 'Parenting\\Parenting_2008-12.csv', 'Parenting\\Parenting_2008-3.csv', 'Parenting\\Parenting_2008-4.csv', 'Parenting\\Parenting_2008-5.csv', 'Parenting\\Parenting_2008-6.csv', 'Parenting\\Parenting_2008-7.csv', 'Parenting\\Parenting_2008-8.csv', 'Parenting\\Parenting_2008-9.csv', 'Parenting\\Parenting_2009-1.csv', 'Parenting\\Parenting_2009-10.csv', 'Parenting\\Parenting_2009-11.csv', 'Parenting\\Parenting_2009-12.csv', 'Parenting\\Parenting_2009-2.csv', 'Parenting\\Parenting_2009-3.csv', 'Parenting\\Parenting_2009-4.csv', 'Parenting\\Parenting_2009-5.csv', 'Parenting\\Parenting_2009-6.csv', 'Parenting\\Parenting_2009-7.csv', 'Parenting\\Parenting_2009-8.csv', 'Parenting\\Parenting_2009-9.csv', 'Parenting\\Parenting_2010-1.csv', 'Parenting\\Parenting_2010-10.csv', 'Parenting\\Parenting_2010-11.csv', 'Parenting\\Parenting_2010-12.csv', 'Parenting\\Parenting_2010-2.csv', 'Parenting\\Parenting_2010-3.csv', 'Parenting

In [3]:
reddit = praw.Reddit(user_agent='collect all data in the subreddit',
                     client_id='your client id', client_secret='your client secret')

Version 5.2.0 of praw is outdated. Version 5.3.0 was released Sunday December 17, 2017.


In [4]:
for f in files[:1]:
    df = pd.read_csv(f)
    ids = df['id'].dropna().astype('str').tolist()
    
    for id in ids:
        submission = reddit.submission(id=id)
        submission.comments.replace_more(limit=None)
        comment_queue = submission.comments[:]  # Seed with top-level
        comments_no_order = [['author','body','created_utc','id','link_id',
                        'parent_id', 'score', 'subreddit_display_name','subreddit_name_prefixed','subreddit_id']]
        
        while comment_queue:
            comment = comment_queue.pop(0)
            comments_no_order.append([str(comment.author),
                                    comment.body, comment.created_utc, comment.id, comment.link_id,
                                    comment.parent_id, comment.score, comment.subreddit.display_name, 
                                    comment.subreddit_name_prefixed, comment.subreddit_id])
            comment_queue.extend(comment.replies)
            
        # save to csv
        with open('Comment/Parenting/' + id + '.csv', 'w', newline="", encoding='utf-8') as f:
            writer = csv.writer(f, delimiter=',')
            writer.writerows(comments_no_order)
            
        # push to s3 bucket
        s3.upload('Comment/Parenting/', 'Comment/Parenting/', id+'.csv')